## Reading the data

In [1]:
import json

newsPath = '../newsCollection/tsCollectionServer/newsData.ts'
with open(newsPath, 'r') as f:
    data = f.read().split('\n')
    data = data[1:-3]
    data = '['+''.join(data)+']'
    # print(data)
    # string ot array
    data = json.loads(data)

In [2]:
for news in data[:3]:
    print(news)
    # print(len(news['data']))

{'source': 'https://example.com', 'heading': 'New Tech Innovations to Watch in 2025', 'newsType': 'Technology', 'datetime': '2025-01-01T10:00:00', 'data': 'The tech world is buzzing with anticipation for what 2025 will bring. From artificial intelligence advancements to quantum computing, the coming year promises major leaps in technology. New gadgets will hit the market, and tech giants are already making waves with their innovative products. One of the most anticipated developments is the integration of AI in daily life, offering personalized experiences in both professional and personal spheres. Experts believe 2025 could be the year technology becomes even more intertwined with everyday routines, leading to smarter homes, better health care, and faster internet speeds.'}
{'source': 'https://technews.com', 'heading': 'AI-Powered Healthcare Devices Revolutionize Diagnosis', 'newsType': 'Technology', 'datetime': '2025-01-01T11:30:00', 'data': 'AI-powered healthcare devices are becomin

## Interacting with the model

In [6]:
import requests
OLLAMA_URL = 'http://localhost:11434/api/'

In [ ]:
def ollamaGenerate(prompt):
    data = {
        "model": "llama3.2:1b",
        "prompt": prompt,
        "stream": False
    }
    # response = requests.post(OLLAMA_URL+'generate', json=data, stream=True)
    # res = ''
    # for line in response.iter_lines():
    #     if line:
    #         # print(line)
    #         print(json.loads(line)['response'])
    #         res = res + json.loads(line)['response']
    # print(res)
    response = requests.post(OLLAMA_URL+'generate', json=data)
    return response
# ollamaGenerate('hi! why is sun hot?')

In [5]:
response = ollamaGenerate('why is sun hot?')
print(response.json())
print(response.json()['response'])

{'model': 'llama3.2:1b', 'created_at': '2025-02-05T13:41:24.867997348Z', 'response': "The Sun is hot because of a process called nuclear fusion. It's the same process that powers the stars and planets in our solar system.\n\nWhen atoms like hydrogen or helium are heated to incredibly high temperatures, they start to overcome their natural attraction to each other and break apart into even smaller particles. This process releases a tremendous amount of energy, which is what makes the Sun so hot.\n\nThe core of the Sun is where this nuclear fusion takes place. Here's a simplified explanation:\n\n1. Hydrogen atoms are heated to temperatures of around 15 million degrees Celsius (27 million degrees Fahrenheit) by the absorption of energy from nearby stars.\n2. At these temperatures, the hydrogen atoms start to fuse together into helium.\n3. This process releases more energy than it takes to create it, which is what makes the Sun shine.\n\nThe amount of energy released per second is enormous

## Creating embeddings using ollama

In [21]:
def ollamaCreateEmbedding(prompt):
    data = {
        # "model": "llama3.2:1b",
        "model": "granite-embedding:278m",
        "prompt": prompt,
    }
    response = requests.post(OLLAMA_URL+'embeddings', json=data)
    return response.json()

In [22]:
res = ollamaCreateEmbedding('hi! why is sun hot?')
print(res)
print(res.keys())
print(len(res['embedding']))

{'embedding': [-0.3245977461338043, 0.4562148451805115, 0.022660046815872192, 0.36946895718574524, -1.098875641822815, -1.2226701974868774, 0.5309364795684814, 1.049145221710205, 0.30311959981918335, -0.6735051870346069, -0.7201023697853088, -0.14930689334869385, 0.4451551139354706, 0.2806950509548187, -0.3460842967033386, 1.155548095703125, 0.024383604526519775, -1.2075285911560059, 0.471038281917572, 0.4591010808944702, -0.3840484023094177, 0.6412433385848999, 0.3155122399330139, 0.6044906973838806, 0.6940922737121582, -0.09141474962234497, -0.36866170167922974, -0.02320799231529236, -0.05188494175672531, 0.16699573397636414, 0.8949435353279114, 1.8402652740478516, 0.26115116477012634, 0.20666739344596863, -0.4728933572769165, 0.08156916499137878, -0.2457943558692932, 1.3272736072540283, -0.4487040638923645, 0.24559283256530762, -0.39699071645736694, 1.8298488855361938, 0.42849287390708923, 1.7056277990341187, -0.6512309312820435, 0.6450209617614746, -0.7532699704170227, 0.1535491049

## Chroma DB

In [15]:
import chromadb
import asyncio
import os
# client = chromadb.Client()

In [ ]:
# client = await chromadb.AsyncHttpClient(host='0.0.0.0', port=8000)
client = chromadb.HttpClient(host='0.0.0.0', port=8005)

ValueError: Could not connect to a Chroma server. Are you sure it is running?

In [3]:
client.heartbeat()
# client.reset()

1738772648554318991

In [4]:
# collection = client.create_collection('news')
collection = client.get_or_create_collection('news')

In [6]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [6]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

{'ids': [['id1', 'id2']], 'distances': [[1.0404008937271816, 1.2430800215233073]], 'embeddings': None, 'metadatas': [[None, None]], 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'included': ['distances', 'documents', 'metadatas']}


In [14]:
# collection.peek()
collection.count()

2